## Collaborative/Personalized Learning

In [1]:
%matplotlib inline
import sys
import os
helpers_path = os.path.abspath('../')
if helpers_path not in sys.path:
    sys.path.append(helpers_path)

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import Kernel

from online_kernels.klms import KLMS
from online_kernels.knlms import KNLMS
from online_kernels.krls import KRLS
from online_kernels.klmsRFF import klms_rff
from online_kernels.knlmsRFF import knlms_rff
from online_kernels.krlsRFF import krls_rff

from data.datasets import load_dodd
from sklearn.utils import shuffle

In [26]:
iteration = 5
# Kernel Dictionary size
D = 200
step_size = 0.8
KLMS_RFF = klms_rff(step_size,D)
num_data=300
# N individuals
N = 10

# L canonical models
L = 3

num_experiments = 1
C = np.ones((L,N))
A = np.zeros((D,L))
u,d,d_true = load_dodd(num_data)
kernel = Kernel(3.73)
sigma = 1/np.sqrt(2*kernel.sigma)
W = (1/sigma) * np.random.normal(size=(2,D))
b = np.random.uniform(0,2*np.pi,(D,1))
h = np.sqrt(2/D) * np.cos(W.T @ u.T + b)
K = []
Y = []
for epoch in tqdm(range(iteration)):

    # Learning A gived fixed Cs (Least Squares)
    B = np.zeros((D,N))
    m = 0
    for i in range(N):
        m_i = np.random.randint(1,10)
        m += m_i
        K_i = np.zeros((D,m_i))
        Y_i = np.zeros((m_i,))
        c_i = C.T[i]
        for samples in range(m_i):
            v = np.random.randint(len(d))
            h_n = h[:,v].reshape((D,1))
            d_n = np.array([d[v]])
            K_i.T[samples] = h_n.reshape((D,))
            Y_i[samples] = d_n
            if samples > 0:
                alpha_in = alpha_out
            else:
                alpha_in = (A @ c_i).reshape((D,1))
            _,alpha_out = KLMS_RFF.train(h_n,d_n,alpha_in)
#             print('alpha',alpha_out)
        K.append(K_i)
        Y.append(Y_i)
    #     Y_i_hat = K_i.T @ A @ c_i

        B.T[i] = alpha_out.reshape((D,))
    A_hat,_,_,_ = np.linalg.lstsq(C.T, B.T,rcond=None) 
    A = A_hat.T
#     print(A)
    # Learning C given the A are fixed
    for i in range(N):
        y_i = Y[i]
        K_i = K[i]
        c_i = C.T[i]
        AK = A.T @ K_i
        AKy = AK @ y_i
        AKKAc = AK @ AK.T @ c_i
        for k in range(L):
            C[k][i] = C[k][i]*((AKy[k] + (AKKAc @ c_i) )/(AKKAc[k] + AKy @ c_i))
print(C)

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 172.58it/s]

[[ 6.82395807e+01 -1.36553639e+04  2.82347593e+07  2.26400458e+06
  -1.97940902e+08  1.88697469e+03  8.92357651e-01 -4.22135024e+13
  -4.41854373e+06  8.99899747e+02]
 [ 6.70904452e+05 -1.32941256e+08  2.34030060e+10 -1.05087497e+11
   1.77967203e+13  9.77689613e+07  1.89911314e+04  6.28254285e+21
  -4.09642497e+11 -2.12715651e+07]
 [ 1.81450733e+06 -6.66090404e+07  6.48152669e+10  3.74815175e+12
   2.76030765e+13  2.12225797e+06 -1.37699534e+05  1.75496937e+21
  -1.89972601e+13 -2.93235059e+05]]


In [14]:
iteration = 1
# N individuals
N = 10
# p features
p = 2
# K canonical models (personalized learning systems)
K = 5
# L 
L = 3
# Kernel Dictionary size
D = 50
num_experiments = 1
C = np.ones((K,N))
A = np.zeros((D,K))
# u,d,d_true = load_dodd(num_data)

# W = (1/sigma) * np.random.normal(size=(2,D))
# b = np.random.uniform(0,2*np.pi,(D,1))
# h = np.sqrt(2/D) * np.cos(W.T @ u.T + b)

for i in tqdm(range(iteration)):
#     v = np.random.randint(len(d))
    
    for edge in range(N):
        # Learning A gived fixed Cs (Least Squares)  
            # Convert c to C
        C_i = np.append(C.T[edge],np.zeros((K*(p-1))))
        for k in range(p-1):
            C_i = np.vstack((C_i,np.roll(np.append(C.T[edge],np.zeros((K*(p-1)))),K*(k+1))))

        # Learn C given the A are fixed (Difficult)
        # This is an iteration process (nonconvex)



        # Calculate a

        # Convert a to A

        # Calculate C
        C = 

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2799.94it/s]


In [69]:
C.shape

(4, 10)